In [ ]:
# Création du dossier WebCrawler contenant l'ensemble des fichiers utiles au fonctionnement de scrapy
!scrapy startproject MangaCrawler

In [ ]:
# ***************** création du projet Manga *********

!cd MangaCrawler && scrapy genspider mangascantrad https://www.japscan.me/

In [ ]:
# ***************** Ajout au fichier items.py & Liste des données qui seront récupérés par manga *********

class MangacrawlerItem(scrapy.Item):
    # define the fields for your item here like:
    title = scrapy.Field()
    img = scrapy.Field()
    rating = scrapy.Field()
    last_chapter = scrapy.Field()
    link = scrapy.Field()
    genres = scrapy.Field()
    published_date = scrapy.Field()
    state = scrapy.Field()
    nb_comments = scrapy.Field()
    pass

In [ ]:
# ***************** Liste des commandes********************

#1. Le titre du manga
response.css('div#main div.card-body h1::text').get()

#2. Le lien de l'image du manga
response.css('div#main div.card-body img').attrib['src']

#3. L'origine du manga
response.css('div#main div.card-body p.mb-2')[1].css('span::text')[1].get()

#4. La description du manga
response.css('div#main div.card-body p.list-group-item.list-group-item-primary.text-justify::text').get()

#5. Le dernier chapitre du manga
manga.css('div#chapters_list div.collapse.show div.chapters_list.text-truncate a::text').get().strip()

#5. Le lien de la page détail du manga
response.request.url

#8. Les genres du manga
response.css('div#main div.card-body p.mb-2')[5].get().split('</span')[1].split('\t')[7].strip()

#6. Le type du manga
response.css('div#main div.card-body p.mb-2')[4].get().split('</span')[1].split('\t')[7].strip()

#6. La date de publication du manga
response.css('div#main div.card-body p.mb-2')[3].get().split('</span')[1].split('\t')[7].strip()

#7. L'état du manga
response.css('div#main div.card-body p.mb-2')[2].get().split('</span')[1].split('\t')[6].strip()


In [ ]:
# ********************** La classe MangascantradSpider **************

import scrapy
from scrapy import Request
from MangaCrawler.items import MangaTypes, MangacrawlerItem, DataBase
import sqlalchemy as db

class MangascantradSpider(scrapy.Spider):
    name = 'mangascantrad'
    allowed_domains = ['www.japscan.me']
    domaine = "https://www.japscan.me"

    # Liste des urls par pages
    start_urls_list = [f'https://www.japscan.me/mangas/{n}' for n in range(1, 31)]
    mangas_url_list = []

    # Création de la base de données
    database = DataBase('database_manga')

    # Creation des tables avec une relation ManyToMany
    database.create_table('mangas_types',
                          id_=db.Integer,
                          name=db.String,
                          )

    database.create_table('mangas',
                          id_=db.Integer,
                          title=db.String,
                          img=db.String,
                          origin=db.String,
                          description=db.String,
                          last_chapter=db.String,
                          link=db.String,
                          type=db.String,
                          genres=db.String,
                          published_date=db.String,
                          state=db.String,
                          )

    database.create_table_relationship('mangas_assoc_types', 'mangas', 'mangas_types')

    def start_requests(self):
        # Récupérer la liste des liens
        for url in self.start_urls_list:
            # appel la fonction addlinks en asynchrone avec la response
            yield Request(url=url, callback=self.addlinks)
        # Récupérer les informations par lien manga
        for url in self.mangas_url_list:
            yield Request(url=url, callback=self.parse_manga)

    def addlinks(self, response):
        items = response.css("div.p-2 p.p-1.text-center")
        for item in items:
            yield self.mangas_url_list.append(self.domaine + item.css("a").attrib['href'])

    def parse_manga(self, response):
        # Récupérer les informations par lien manga
        manga = response
        item = MangacrawlerItem()

        # Titre manga
        try:
            item['title'] = manga.css('div#main div.card-body h1::text').get()
        except:
            item['title'] = 'None'

        # Image manga
        try:
            item['img'] = self.domaine + manga.css('div#main div.card-body img').attrib['src']
        except:
            item['img'] = 'None'

        # Origin manga
        try:
            item['origin'] = manga.css('div#main div.card-body p.mb-2')[1].css('span::text')[1].get()
        except:
            item['origin'] = manga.css('div#main div.card-body p.mb-2')[2].css('span::text')[1].get()

        # description manga
        try:
            item['description'] = manga.css('div#main div.card-body p.list-group-item.list-group-item-primary.text-justify::text').get()
        except:
            item['description'] = 'None'

        # Dernier chapitre manga
        try:
            item['last_chapter'] = manga.css('div#chapters_list div.collapse.show div.chapters_list.text-truncate a::text').get().strip()
        except:
            item['last_chapter'] = 'None'

        # Lien manga
        try:
            item['link'] = manga.request.url
        except:
            item['link'] = 'None'

        # Type du manga
        try:
            item['type'] = 'None'
            for i in range (1, 8):
                if manga.css('div#main div.card-body p.mb-2')[i].get().split(':')[0].split('>')[2] == 'Type(s)':
                    item['type'] = manga.css('div#main div.card-body p.mb-2')[i].get().split('</span')[1].split('\t')[7].strip()
        except:
            item['type'] = 'None'

        # genres manga
        try:
            item['genres'] = 'None'
            for i in range(1, 8):
                if manga.css('div#main div.card-body p.mb-2')[i].get().split(':')[0].split('>')[2] == 'Genre(s)':
                    item['genres'] = manga.css('div#main div.card-body p.mb-2')[i].get().split('</span')[1].split('\t')[7].strip()
        except:
            item['genres'] = 'None'

        # Date du manga
        try:
            item['published_date'] = 'None'
            for i in range(1, 8):
                if manga.css('div#main div.card-body p.mb-2')[i].get().split(':')[0].split('>')[2] == 'Date Sortie':
                    item['published_date'] = manga.css('div#main div.card-body p.mb-2')[i].get().split('</span')[1].split('\t')[7].strip()
        except:
            item['published_date'] = 'None'

        # Etat manga
        try:
            stateM = manga.css('div#main div.card-body p.mb-2')[2].get().split('</span')[1].split('\t')[6].strip()
            item['state'] = stateM if stateM != '' else manga.css('div#main div.card-body p.mb-2')[3].get().split('</span')[1].split('\t')[6].strip()
        except:
            item['state'] = "None"

        # Ajouter dans la base de données
        self.database.add_row('mangas',
                              title=item['title'],
                              img=item['img'],
                              origin=item['origin'],
                              description=item['description'],
                              last_chapter=item['last_chapter'],
                              link=item['link'],
                              type=item['type'],
                              genres=item['genres'],
                              published_date=item['published_date'],
                              state=item['state']
                              )
        yield item



In [31]:
# Exécution de scrapy
!cd MangaCrawler/MangaCrawler/spiders && scrapy crawl mangascantrad -o mangascantrad.csv

*********Table : 'mangas_types' are created succesfully************
*********Table : 'mangas' are created succesfully************
*********Table Relationship : 'mangas_assoc_types' are created succesfully************
2022-10-20 19:57:27 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: MangaCrawler)
2022-10-20 19:57:27 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.9.2 (default, Feb 28 2021, 17:03:44) - [GCC 10.2.1 20210110], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.0-19-amd64-x86_64-with-glibc2.31
2022-10-20 19:57:27 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'MangaCrawler',
 'NEWSPIDER_MODULE': 'MangaCrawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['MangaCrawler.spiders']}
2022-10-20 19:57:27 [py.warnings] WARNING: /home/yannis/.local/lib/python3.9/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is

2022-10-20 19:57:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/100-jours-avant-ta-mort/> (referer: None)
2022-10-20 19:57:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japscan.me/manga/100-jours-de-reve/> (referer: None)
Row id added
2022-10-20 19:57:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/100-demons-of-love/>
{'description': 'Coco, 15 ans est une jeune fille qui a la particularité de '
                'voir les fantômes. Mais un jour, en allant au lycée elle '
                'voit pour la première fois deux démons, deux frères '
                'terriblement beaux : Reo et Toa. Alors qu’elle se fait '
                'agresser par l’esprit d’un chien noir, Coco, pleine de pitié '
                'pour cet esprit, dégage un pouvoir spécial qui va '
                'intéresser les deux frères démons. Quel est ce pouvoir qui '
                'habite la jeune et qui attire les démons?',
 '

Row id added
2022-10-20 19:57:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/100-gokuama-kareshi/>
{'description': "Petite anthologie d'histoires écrites par différentes "
                'auteures.\r\n'
                '1. Kodomo Jya, Naino - Ririko Yoshioka\r\n'
                "Nano et Ryou sont deux amis d'enfances très proche l'un "
                "l'autre. Quand ils était jeunes Nano promis à Ryou de le "
                'protéger de tous les dangers comme le ferait une grande sœur. '
                'Mais quelques années plus tard, les rôles se sont peut-être '
                'inversés.\r\n'
                '2. Natsu ga Chikai - Ayako Kimura\r\n'
                'Asamizu aime Tsuji Hideto et, afin de pouvoir lui parler tous '
                "les jours, elle viole délibérément les règlements de l'école "
                'afin de lui convenir.\r\n'
                '3. HOLE - Kirameku Sekai - Rie Takahashi\r\n'
                "Il n'arrêtait pas d

Row id added
2022-10-20 19:57:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/100-nin-no-eiyuu-o-sodateta-saikyou-yogensha-wa-boukensha-ni-natte-mo-sekaijuu-no-deshi-kara-shitawarete-masu/>
{'description': 'Élevant une centaine de héros et mettant fin à la guerre avec '
                'un dieu maléfique, Eisen a sauvé le monde. Après avoir été '
                'gravement blessé en protégeant les héros, il est parti dans '
                "l'Autre Monde pour se rétablir. Pendant ce temps, les héros "
                'ont fait usage de leurs capacités pour contribuer grandement '
                'au développement du monde. Désireux de rencontrer à nouveau '
                "le maître qu'ils admiraient, ils lui firent un compte rendu "
                'détaillé de leurs réalisations. En lisant leurs lettres, '
                'Eisen prend la décision de partir et de rendre visite à '
                "chacun d'entre eux au cours de son voyage à travers le p

Row id added
2022-10-20 19:57:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/15-minutes-avant-qu-ils-ne-sortent-ensemble/>
{'description': "Une adorable histoire de deux amis d'enfance qui commencent à "
                'se fréquenter. En terminale, aucun moment ne peut être perdu, '
                "car ils ont peu de temps restant avant l'université. Tout "
                'commence par "Yuuki, séparons nous.".',
 'genres': 'School Life, Comédie, Romance, Slice Of Life, Webcomic',
 'img': 'https://www.japscan.me/imgs/mangas/15-minutes-avant-qu-ils-ne-sortent-ensemble.jpg',
 'last_chapter': "15 Minutes Avant Qu'ils Ne Sortent Ensemble 22.2 VF: Un "
                 'rencard de Noël sérieux (2)',
 'link': 'https://www.japscan.me/manga/15-minutes-avant-qu-ils-ne-sortent-ensemble/',
 'origin': 'Japon',
 'published_date': '2019',
 'state': 'En Cours',
 'title': "Manga 15 Minutes Avant Qu'ils Ne Sortent Ensemble",
 'type': 'Seinen'}
Row id added
2022-10-20 1

...

Row id added
2022-10-20 19:59:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/bouncer-mizuta-makoto/>
{'description': 'On suit le parcours de Jouichirou Shishido, un gros loser de '
                'base, qui une nuit, va tomber sur un groupe de garde de corps '
                'privé (Tokyo Fist) qui changera sa vie à tout jamais.',
 'genres': 'Drame, Action, Aventure',
 'img': 'https://www.japscan.me/imgs/mangas/bouncer-mizuta-makoto.jpg',
 'last_chapter': 'Bouncer (Mizuta Makoto) 67 VF',
 'link': 'https://www.japscan.me/manga/bouncer-mizuta-makoto/',
 'origin': 'Japon',
 'published_date': '2014',
 'state': 'En Cours',
 'title': 'Manga Bouncer (Mizuta Makoto)',
 'type': 'Shōnen, Seinen'}
Row id added
2022-10-20 19:59:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/bounder/>
{'description': "Le peuple souffre sous l'oppression de l'Empereur Qin, le "
                'premier à avoir unifié la Chine. Yu, un jeune orphelin, a

Row id added
2022-10-20 19:59:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/boy-of-the-female-wolf/>
{'description': "Bien qu'Eun-Soo soit une fille, elle vit en tant que garçon "
                'depuis des années, au point que personne ne sait la vérité '
                'sur son sexe. De plus, elle adore se battre, ce qui ne plait '
                "absolument pas à sa grand-mère, avec qui elle vit jusqu'à ce "
                "qu'elle décède.\r\n"
                "À ce moment-là, Eun-Soo est obligée d'aller vivre à Séoul, "
                "chez sa mère qu'elle déteste pour l'avoir abandonnée.\r\n"
                "Une situation d'autant plus difficile que cette dernière "
                "s'est remariée et vit avec Gyu-Hyun, le fils de son nouveau "
                'mari.\r\n'
                '\r\n'
                "Et bien qu'elle avait promis à sa grand-mère de ne plus se "
                "battre, dès son arrivée, Eun-Soo s'attire des problèmes 

Row id added
2022-10-20 19:59:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/boys-n-girl/>
{'description': "Aira est ravie à l'idée de pouvoir enfin emménager dans une "
                'nouvelle maison avec son père, gardien de dortoir au lycée '
                "Saiko. Mais ce que son père ne lui a pas dit, c'est que leur "
                'nouvelle maison se trouve pas loin du dortoir des pires '
                'garçons de fameux lycée ! Aria se voit donc obligée de vivre '
                "dans l'enceinte d'un lycée et de cohabiter avec ces voisins "
                "plus que turbulents, dont leur leader, Tennoji, n'aura comme "
                "seul objectif de rendre la vie d'Aira infernale.",
 'genres': 'School Life, Sport, Comédie, Ecole, Romance',
 'img': 'https://www.japscan.me/imgs/mangas/boys-n-girl.jpg',
 'last_chapter': "Boys'n Girl 13 VF",
 'link': 'https://www.japscan.me/manga/boys-n-girl/',
 'origin': 'Japon',
 'published_date': '1997',

Row id added
2022-10-20 19:59:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/bravely-default-flying-fairy/>
{'description': 'Tiz Arrior est une jeune berger vivant dans le petit village '
                'de Norende avec son frère Til. Alors qu’ils menaient une vie '
                'paisible, leur village s’effondre et malgré tous ses efforts, '
                'Tiz n’arrive pas à sauver son petit frère. Secouru de '
                'justesse par un village voisin, Tiz se trouve être le seul '
                'survivant et son village.',
 'genres': 'Fantastique, Drame, Surnaturel, Action, Magie, Aventure, Romance, '
           'Tragédie',
 'img': 'https://www.japscan.me/imgs/mangas/bravely-default-flying-fairy.jpg',
 'last_chapter': 'Bravely Default - Flying Fairy 6.2 VF',
 'link': 'https://www.japscan.me/manga/bravely-default-flying-fairy/',
 'origin': 'Japon',
 'published_date': '2015',
 'state': 'En Pause',
 'title': 'Manga Bravely Default - Flying F

Row id added
2022-10-20 19:59:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.japscan.me/manga/bremen/>
{'description': 'Romio Kasuga un jeune garçon en fugue doué pour la "baston" '
                'qui a pour ambition de devenir très connu et surtout très '
                'riche. Sur son chemin, il croise Reiji Hino un jeune '
                'guitariste fou de rock qui veut percer. Ils vont partir '
                'ensemble pour Tokyo et former un groupe de rock nommé '
                '"Bremen"...',
 'genres': 'Drame, Action, Tranche De Vie, Comédie, Aventure, Science-fiction, '
           'Gender Bender, Musique, Crossdressing',
 'img': 'https://www.japscan.me/imgs/mangas/bremen.jpg',
 'last_chapter': 'Bremen Volume 9 VF',
 'link': 'https://www.japscan.me/manga/bremen/',
 'origin': 'Japon',
 'published_date': '2000',
 'state': 'Terminé',
 'title': 'Manga Bremen',
 'type': 'Shōnen'}
2022-10-20 19:59:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.japsc

In [32]:
# ***************** Afficher les données csv avec pandas *******************

import pandas as pd

df = pd.read_csv('MangaCrawler/MangaCrawler/spiders/mangascantrad.csv')
df

,description,genres,img,last_chapter,link,origin,published_date,state,title,type
0,Un groupe de jeunes gens tourmentés par des fa...,"Psychologique, Horreur, Webtoon",https://www.japscan.me/imgs/mangas/0-0-mhz.jpg,0.0 Mhz 4 VF: Routes séparées,https://www.japscan.me/manga/0-0-mhz/,Corée Du Sud,2012,En Cours,Manhwa 0.0 Mhz,Seinen
1,"Coco, 15 ans est une jeune fille qui a la part...","Fantastique, Drame, Mystère, School Life, Surn...",https://www.japscan.me/imgs/mangas/100-demons-...,100 Demons Of Love 28 VF: FIN,https://www.japscan.me/manga/100-demons-of-love/,Japon,2012,Terminé,Manga 100 Demons Of Love,Shōjo
2,"Il y a de cela plus de mille ans, le monde éta...","Fantastique, Drame, Mystère, Surnaturel, Démon...",https://www.japscan.me/imgs/mangas/07-ghost.jpg,07 Ghost Volume 17 VF: FIN,https://www.japscan.me/manga/07-ghost/,Japon,2005,Terminé,Manga 07 Ghost,Josei
3,Hiroshi a trouvé un téléphone portable cassé d...,"Fantastique, Drame, School Life, Tranche De Vi...",https://www.japscan.me/imgs/mangas/090-eko-to-...,090 Eko To Issho 31 VF,https://www.japscan.me/manga/090-eko-to-issho/,Japon,2006,En Pause,Manga 090 Eko To Issho,Shōnen
4,"A l’époque maudite du collègue, Ai Maekawa éta...","Drame, School Life, Tranche De Vie, Comédie, R...",https://www.japscan.me/imgs/mangas/100-doubt.jpg,100% Doubt Volume 6 VF: FIN,https://www.japscan.me/manga/100-doubt/,Japon,2000,Terminé,Manga 100% Doubt,Shōjo
...,...,...,...,...,...,...,...,...,...,...
813,A venir...,"Drame, Mystère, Psychologique, Action, Militai...",https://www.japscan.me/imgs/mangas/breaking-th...,"Breaking Through The Clouds 1 VF: Jiang Ting, ...",https://www.japscan.me/manga/breaking-through-...,Chine,None,En Cours,Manhua Breaking Through The Clouds,"Josei, Shounen"
814,"La vie d’Amir, 20 ans, est bouleversée le jour...","Drame, Mariage, Tranche De Vie, Comédie, Roman...",https://www.japscan.me/imgs/mangas/bride-stori...,Bride Stories 69 VF: Retrouvailles,https://www.japscan.me/manga/bride-stories/,Japon,2008,En Cours,Manga Bride Stories,Seinen
815,Romio Kasuga un jeune garçon en fugue doué pou...,"Drame, Action, Tranche De Vie, Comédie, Aventu...",https://www.japscan.me/imgs/mangas/bremen.jpg,Bremen Volume 9 VF,https://www.japscan.me/manga/bremen/,Japon,2000,Terminé,Manga Bremen,Shōnen
816,Ne vous êtes-vous jamais dit que le vrai amour...,"Fantastique, Drame, Comédie, Historique, Roman...",https://www.japscan.me/imgs/mangas/bring-the-l...,Bring The Love 63 VF,https://www.japscan.me/manga/bring-the-love/,Corée Du Sud,2020,En Cours,Manhwa Bring The Love,Shōjo
